<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=aa8b6f2951889f628bab9cec41968dbd1c1de181c157f12063cd048d03fba5f8
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-29 10:20:58
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.37 C
-------------------
Today PnL: 63.15 K (0.46%)
Current PnL: -22.72 L (-15.28%)
CY Booked + Current PnL: -11.28 L (-7.59%)
-------------------
Total profit:  98.68 K
Total loss:  -23.71 L
-------------------
Total Booked + Current PnL: 15.70 L (12.9%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.36%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.59 L (64.73%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.09%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,94.88,40.0,M-SC,2.47,83964.0,-16813.0,16902.0,0.94,-16.68,20.13,0.09,245.0,-0.99,0.61,10.31,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.48,47.0,H-MC,3.62,188960.0,24290.0,19047.0,-0.03,14.75,10.08,26.31,89.0,1.28,1.38,23.91,X40N,NTT,AC
50,NATIONALUM,244.55,-45.32,53.0,H-MC,11.89,108586.0,8082.0,21022.0,2.14,8.04,19.36,28.96,79.0,0.38,0.79,44.74,MH,ATH,METALS
42,ITC,452.00,-38.34,48.0,H-LC,1.90,199047.0,-1091.0,21975.0,0.48,-0.55,11.04,10.44,4.0,-0.05,1.45,4.99,X40,NTT,FMCG
57,RELIANCE,1533.00,-13.66,48.0,H-LC,3.94,216200.0,5834.0,22939.0,0.60,2.77,10.61,13.68,37.0,0.25,1.58,19.75,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-177.27,70.0,M-SC,6.60,89478.0,-12732.0,106121.0,8.21,-12.46,118.60,91.37,208.0,-0.12,0.65,45.25,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-45.67,45.0,L-SC,40.38,80141.0,-33408.0,73457.0,3.24,-29.42,91.66,35.27,268.0,-0.45,0.59,96.15,XR,NTT,HOTELS
32,HINDZINC,730.22,23.40,58.0,M-LC,2.94,201253.0,-3823.0,116385.0,3.02,-1.86,57.83,54.89,52.0,-0.03,1.47,21.78,X5K,ATH,METALS
10,BANDHANBNK,400.00,-14.68,42.0,H-SC,4.14,213489.0,-65070.0,322518.0,2.77,-23.36,151.07,92.42,151.0,-0.20,1.56,23.38,XY24,NTT,BANKS
11,BANKINDIA,190.00,-29.19,57.0,H-MC,8.49,183837.0,4029.0,108390.0,2.75,2.24,58.96,62.52,88.0,0.04,1.34,37.09,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIGOPNTS,1408.00,100.89,29.0,M-SC,1.53,131700.0,-42859.0,42895.0,-2.61,-24.55,32.57,0.02,221.0,-1.00,0.96,14.56,OX40N,NTT,PAINTS
71,TANLA,1963.11,-22.79,48.0,H-SC,17.36,206948.0,-63813.0,393760.0,-2.45,-23.57,190.27,121.86,133.0,-0.16,1.51,61.89,AR,ATH,IT
13,BERGEPAINT,680.00,-23.15,33.0,H-MC,4.23,206672.0,-20667.0,68718.0,-2.14,-9.09,33.25,21.14,106.0,-0.30,1.51,16.49,XY24,NTT,PAINTS
55,RAJOOENG,143.10,-44.32,22.0,H-SC,14.18,84310.0,-18190.0,58789.0,-1.41,-17.75,69.73,39.61,114.0,-0.31,0.62,0.00,AR,ATH,MISC
26,GREENPANEL,537.00,197.80,41.0,M-SC,1.03,133004.0,-50074.0,129054.0,-1.25,-27.35,97.03,43.14,230.0,-0.39,0.97,25.11,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.0,-45.46,29.0,M-SC,1.4,92205.0,706.0,78042.0,-0.26,0.77,84.64,86.07,223.0,0.01,0.67,33.9,XR,NTT,DURABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,51.0,H-SC,10.33,121959.0,-20520.0,40039.0,2.24,-14.40,32.83,13.70,163.0,-0.51,0.89,43.95,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,94.88,40.0,M-SC,2.47,83964.0,-16813.0,16902.0,0.94,-16.68,20.13,0.09,245.0,-0.99,0.61,10.31,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,51.0,H-SC,1.54,221139.0,-48528.0,84851.0,-0.35,-18.00,38.37,13.47,138.0,-0.57,1.62,11.85,XY24,NTT,PAINTS
18,CERA,9475.0,-22.67,36.0,H-SC,1.70,140875.0,-35028.0,77045.0,-0.88,-19.91,54.69,23.89,149.0,-0.45,1.03,21.66,OX40N,NTT,CERAMICS
66,SIS,528.0,2073.58,39.0,H-SC,4.02,87707.0,-23325.0,46932.0,-0.36,-21.01,53.51,21.26,156.0,-0.50,0.64,18.50,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.95,57.0,M-MC,10.76,229939.0,4977.0,162682.0,1.92,2.21,70.75,74.53,192.0,0.03,1.68,35.18,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,9.48,47.0,H-MC,3.62,188960.0,24290.0,19047.0,-0.03,14.75,10.08,26.31,89.0,1.28,1.38,23.91,X40N,NTT,AC
84,VOLTAS,1530.00,-3.46,42.0,H-MC,2.83,202575.0,10833.0,26922.0,0.77,5.65,13.29,19.69,99.0,0.40,1.48,12.83,XY25,NTT,AC
17,CAMS,5211.76,-5.83,44.0,H-SC,1.79,107016.0,5012.0,38911.0,0.58,4.91,36.36,43.06,122.0,0.13,0.78,22.87,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,54.0,M-LC,7.04,155121.0,14281.0,30218.0,1.75,10.14,19.48,31.59,66.0,0.47,1.13,39.31,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-45.32,53.0,H-MC,11.89,108586.0,8082.0,21022.0,2.14,8.04,19.36,28.96,79.0,0.38,0.79,44.74,MH,ATH,METALS
11,BANKINDIA,190.00,-29.19,57.0,H-MC,8.49,183837.0,4029.0,108390.0,2.75,2.24,58.96,62.52,88.0,0.04,1.34,37.09,XR,NTT,BANKS
37,INDIAMART,4850.92,-54.83,28.0,H-SC,10.00,125928.0,2592.0,131167.0,0.39,2.10,104.16,108.45,119.0,0.02,0.92,24.92,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-45.46,29.0,M-SC,1.40,92205.0,706.0,78042.0,-0.26,0.77,84.64,86.07,223.0,0.01,0.67,33.90,XR,NTT,DURABLES
86,WIPRO,420.00,-15.61,38.0,M-LC,5.46,148440.0,-2505.0,111953.0,1.56,-1.66,75.42,72.51,53.0,-0.02,1.08,4.26,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.03,30.0,H-LC,6.53,206976.0,-44792.0,98003.0,0.39,-17.79,47.35,21.13,27.0,-0.46,1.51,11.33,X40,ATH,PAINTS
83,VBL,671.64,-21.10,30.0,H-LC,8.53,284420.0,-31422.0,144087.0,0.24,-9.95,50.66,35.67,5.0,-0.22,2.08,2.53,X40N,ATH,FMCG
74,TCS,4476.75,-30.93,32.0,H-LC,12.14,274762.0,-71194.0,146063.0,0.82,-20.58,53.16,21.64,1.0,-0.49,2.01,0.82,X40,ATH,IT
30,HCLTECH,1943.91,-4.09,34.0,H-LC,10.00,219172.0,-22752.0,86025.0,0.05,-9.40,39.25,26.15,8.0,-0.26,1.60,3.57,X40,ATH,IT
44,JIOFIN,387.00,-5.20,34.0,H-LC,12.82,257656.0,-9709.0,75158.0,1.27,-3.63,29.17,24.48,48.0,-0.13,1.88,49.34,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.51,41.0,H-LC,1.58,229500.0,-16651.0,95862.0,0.49,-6.76,41.77,32.18,16.0,-0.17,1.68,26.84,X200,ATH,IT
42,ITC,452.0,-38.34,48.0,H-LC,1.90,199047.0,-1091.0,21975.0,0.48,-0.55,11.04,10.44,4.0,-0.05,1.45,4.99,X40,NTT,FMCG
65,SIEMENS,4671.5,-2.61,44.0,H-LC,2.39,156440.0,-29655.0,77141.0,1.05,-15.94,49.31,25.51,15.0,-0.38,1.14,15.28,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-13.66,48.0,H-LC,3.94,216200.0,5834.0,22939.0,0.60,2.77,10.61,13.68,37.0,0.25,1.58,19.75,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-21.84,44.0,H-LC,4.34,208596.0,4096.0,37318.0,1.55,2.00,17.89,20.25,10.0,0.11,1.52,10.91,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.93,32.0,H-LC,12.14,274762.0,-71194.0,146063.0,0.82,-20.58,53.16,21.64,1.0,-0.49,2.01,0.82,X40,ATH,IT
80,UNITDSPR,1644.00,-14.37,44.0,H-LC,7.20,224460.0,-9684.0,58315.0,0.59,-4.14,25.98,20.77,86.0,-0.17,1.64,2.50,X40N,NTT,BREWERIES
83,VBL,671.64,-21.10,30.0,H-LC,8.53,284420.0,-31422.0,144087.0,0.24,-9.95,50.66,35.67,5.0,-0.22,2.08,2.53,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.09,34.0,H-LC,10.00,219172.0,-22752.0,86025.0,0.05,-9.40,39.25,26.15,8.0,-0.26,1.60,3.57,X40,ATH,IT
42,ITC,452.00,-38.34,48.0,H-LC,1.90,199047.0,-1091.0,21975.0,0.48,-0.55,11.04,10.44,4.0,-0.05,1.45,4.99,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.61,44.0,H-LC,2.39,156440.0,-29655.0,77141.0,1.05,-15.94,49.31,25.51,15.0,-0.38,1.14,15.28,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.34,48.0,H-LC,1.90,199047.0,-1091.0,21975.0,0.48,-0.55,11.04,10.44,4.0,-0.05,1.45,4.99,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.03,30.0,H-LC,6.53,206976.0,-44792.0,98003.0,0.39,-17.79,47.35,21.13,27.0,-0.46,1.51,11.33,X40,ATH,PAINTS
19,CIPLA,1795.00,-21.84,44.0,H-LC,4.34,208596.0,4096.0,37318.0,1.55,2.00,17.89,20.25,10.0,0.11,1.52,10.91,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-13.66,48.0,H-LC,3.94,216200.0,5834.0,22939.0,0.60,2.77,10.61,13.68,37.0,0.25,1.58,19.75,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.93,32.0,H-LC,12.14,274762.0,-71194.0,146063.0,0.82,-20.58,53.16,21.64,1.0,-0.49,2.01,0.82,X40,ATH,IT
40,INFY,2275.00,-19.96,43.0,H-LC,8.17,311875.0,-1013.0,172685.0,1.06,-0.32,55.37,54.87,3.0,-0.01,2.28,6.30,X40,BTT,IT
42,ITC,452.00,-38.34,48.0,H-LC,1.90,199047.0,-1091.0,21975.0,0.48,-0.55,11.04,10.44,4.0,-0.05,1.45,4.99,X40,NTT,FMCG
83,VBL,671.64,-21.10,30.0,H-LC,8.53,284420.0,-31422.0,144087.0,0.24,-9.95,50.66,35.67,5.0,-0.22,2.08,2.53,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,47.0,H-LC,7.67,250531.0,-11088.0,130301.0,0.89,-4.24,52.01,45.57,7.0,-0.09,1.83,6.60,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,45.0,L-SC,40.38,80141.0,-33408.0,73457.0,3.24,-29.42,91.66,35.27,268.0,-0.45,0.59,96.15,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-177.27,70.0,M-SC,6.60,89478.0,-12732.0,106121.0,8.21,-12.46,118.60,91.37,208.0,-0.12,0.65,45.25,XY25,NTT,FINANCE
49,MASFIN,398.61,-20.51,37.0,H-SC,12.31,90465.0,-7515.0,29121.0,-0.36,-7.67,32.19,22.05,152.0,-0.26,0.66,31.50,XR,ATH,FINANCE
50,NATIONALUM,244.55,-45.32,53.0,H-MC,11.89,108586.0,8082.0,21022.0,2.14,8.04,19.36,28.96,79.0,0.38,0.79,44.74,MH,ATH,METALS
78,UJJIVANSFB,60.00,92.53,51.0,H-SC,10.33,121959.0,-20520.0,40039.0,2.24,-14.40,32.83,13.70,163.0,-0.51,0.89,43.95,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1193.46,65.0,M-SC,14.03,175883.0,-5260.0,50725.0,2.70,-2.90,28.84,25.10,235.0,-0.10,1.29,40.95,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.0,-177.27,70.0,M-SC,6.60,89478.0,-12732.0,106121.0,8.21,-12.46,118.60,91.37,208.0,-0.12,0.65,45.25,XY25,NTT,FINANCE
81,VAIBHAVGBL,521.0,51.52,48.0,H-SC,5.75,131418.0,-51357.0,162433.0,0.23,-28.10,123.60,60.77,125.0,-0.32,0.96,17.71,XR,NTT,JEWELLERY
60,SAIL,228.0,45.95,57.0,M-MC,10.76,229939.0,4977.0,162682.0,1.92,2.21,70.75,74.53,192.0,0.03,1.68,35.18,XY24,BTT,STEEL
7,ATULAUTO,844.0,3527.03,47.0,M-SC,7.42,113138.0,-34230.0,86041.0,0.31,-23.23,76.05,35.16,236.0,-0.40,0.83,15.56,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.27
1,25,43.70
2,50,74.91


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.78
LC    32.54
MC    23.63
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.37
X40      14.78
X40N     11.39
XR       11.14
XY25     10.19
AR        8.26
OX40N     7.85
X200      1.68
MH        1.61
X5K       1.47
SR        1.21
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.27
H-LC    26.00
H-MC    20.67
M-SC    13.04
M-LC     5.48
M-MC     2.66
L-SC     1.47
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.03,-6.80,42.05
IT,12.79,-20.14,85.64
FINANCE,10.09,-18.73,70.52
MISC,6.97,-20.44,87.41
BANKS,6.21,-17.26,78.95
PAINTS,5.60,-20.46,38.42
ELECTRICAL,5.50,-9.47,47.87
INSURANCE,3.77,-7.27,47.37
AC,3.57,0.90,15.69


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3225060.0,22
XR,1385529.0,14
AR,1200738.0,9
X40,825584.0,10
X40N,645813.0,9
OX40N,570906.0,10
XY25,444307.0,7
SR,256111.0,2
X5K,116385.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3790757.0,29
M-SC,1459986.0,17
H-MC,1274954.0,15
H-LC,1259802.0,15
M-LC,399052.0,4
M-MC,313764.0,2
L-SC,259852.0,3
L-MC,61233.0,1
L-LC,39444.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1247947.0      6
M-SC       XY24       836300.0      7
H-SC       AR         822889.0      5
           XR         806981.0      7
H-MC       XY24       581177.0      4
H-LC       X40        524751.0      5
H-SC       X40N       319788.0      4
M-MC       XY24       313764.0      2
H-SC       OX40N      265514.0      4
           SR         256111.0      2
H-LC       X40N       239720.0      3
H-MC       X40        223782.0      4
H-LC       AR         207442.0      2
H-MC       XY25       181873.0      2
L-SC       XR         170962.0      2
M-LC       XY24       170714.0      2
M-SC       AR         170407.0      2
           OX40N      144097.0      4
           XR         126010.0      2
H-LC       XY25       116869.0      3
M-LC       X5K        116385.0      1
           XR         111953.0      1
H-MC       XR         108390.0      1
M-SC       XY25       106121.0      1
H-LC       X200        95862.0      1
L-SC       OX40N       88890.0      1
H-MC       X40N        86305.0      2
M-SC       X40         77051.0      1
H-LC       XY24        75158.0      1
H-MC       OX40N       72405.0      1
H-SC       MH          71527.0      1
L-MC       XR          61233.0      1
L-LC       XY25        39444.0      1
H-MC       MH          21022.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 52.0 seconds
